In [81]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

# from statsmodels.graphics.tsaplots import plot_acf
# from statsmodels.tsa.stattools import adfuller
# from scipy.ndimage.interpolation import shift

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.ensemble import GradientBoostingClassifier
%matplotlib inline

In [2]:
# Versões dos pacotes usados no notebook

%reload_ext watermark
%watermark -a "César S. Hooper/ Novembro 2022" --iversions
!python --version

Author: César S. Hooper/ Novembro 2022

pandas    : 1.2.4
matplotlib: 3.3.4
cv2       : 4.5.4-dev
numpy     : 1.19.5

Python 3.8.8


In [9]:
path = 'C:/Users/Cesar Hooper/Desktop/github/bolsa_chile/resumen.csv'

In [96]:
datos = pd.read_excel('micro emini ES 08-07 18_43.xls')
datos2 = pd.read_excel('micro emini ES 08-12 17_47.xls')
datos3 = pd.read_excel('micro emini ES 08-13 17_10.xls')
datos4 = pd.read_excel('micro emini ES 08-14 18_48.xls')
datos5 = pd.read_excel('micro emini ES 08-15 19_08.xls')
datos6 = pd.read_excel('micro emini ES 08-19 17_41.xls')

In [98]:
datos6.head()

,hora,accion,C,L,L.1,L.2,L.3,CCI5,CCI30,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,21:12,NaN,3894.50,I,L,I,I,24,92,NaN,NaN,NaN,NaN,NaN,NaN
1,21:13,NaN,3893.25,S,L,I,I,-1,89,-1.25,NaN,NaN,NaN,NaN,NaN
2,21:14,NaN,3894.00,S,L,I,I,9,90,-0.50,NaN,NaN,NaN,NaN,NaN
3,21:15,NaN,3894.25,I,L,I,I,18,91,-0.25,NaN,NaN,NaN,NaN,NaN
4,21:16,NaN,3893.75,I,L,I,I,3,90,-0.75,NaN,NaN,NaN,NaN,NaN


In [111]:
datos5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   592 non-null    object 
 1   Unnamed: 1   14 non-null     object 
 2   Unnamed: 2   592 non-null    float64
 3   Unnamed: 3   592 non-null    object 
 4   Unnamed: 4   592 non-null    object 
 5   Unnamed: 5   592 non-null    object 
 6   Unnamed: 6   592 non-null    object 
 7   Unnamed: 7   592 non-null    int64  
 8   Unnamed: 8   592 non-null    int64  
 9   Unnamed: 9   303 non-null    object 
 10  Unnamed: 10  460 non-null    float64
 11  Unnamed: 11  60 non-null     float64
 12  Unnamed: 12  3 non-null      float64
 13  Unnamed: 13  3 non-null      float64
 14  Unnamed: 14  241 non-null    object 
dtypes: float64(5), int64(2), object(8)
memory usage: 69.5+ KB


In [24]:
col = [];
for k in range(1, 16):
    col.append('x' + str(k));
    
col = pd.Series(col);    
col

0      x1
1      x2
2      x3
3      x4
4      x5
5      x6
6      x7
7      x8
8      x9
9     x10
10    x11
11    x12
12    x13
13    x14
14    x15
dtype: object

In [25]:
datos.columns = col
datos.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,07:10,NaN,3914.25,I,I,L,S,-113,84,NaN,NaN,NaN,NaN,NaN,NaN
1,07:11,NaN,3915.25,I,I,L,S,-67,86,NaN,NaN,NaN,NaN,NaN,NaN
2,07:12,NaN,3915.75,I,I,L,S,-56,87,BUY,NaN,NaN,NaN,NaN,NaN
3,07:13,NaN,3915.50,I,I,L,S,-56,86,-0.25,NaN,NaN,NaN,NaN,NaN
4,07:14,NaN,3916.00,I,I,L,S,-45,87,0.25,NaN,NaN,NaN,NaN,NaN


In [66]:
dataset = datos[['x1', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x15']]
dataset.head()

,x1,x3,x4,x5,x6,x7,x8,x9,x15
0,07:10,3914.25,I,I,L,S,-113,84,NaN
1,07:11,3915.25,I,I,L,S,-67,86,NaN
2,07:12,3915.75,I,I,L,S,-56,87,NaN
3,07:13,3915.50,I,I,L,S,-56,86,NaN
4,07:14,3916.00,I,I,L,S,-45,87,NaN


In [67]:
colnew = [0 if k == 'buena' else 1 for k in dataset['x15']]       
colnew = pd.Series(colnew);

In [68]:
dataset['target'] = colnew;

<ipython-input-68-9714a45d117f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['target'] = colnew;


In [69]:
dataset = dataset.drop('x15', axis=1);

In [70]:
dataset.head()

,x1,x3,x4,x5,x6,x7,x8,x9,target
0,07:10,3914.25,I,I,L,S,-113,84,1
1,07:11,3915.25,I,I,L,S,-67,86,1
2,07:12,3915.75,I,I,L,S,-56,87,1
3,07:13,3915.50,I,I,L,S,-56,86,1
4,07:14,3916.00,I,I,L,S,-45,87,1


In [80]:
x_hora = dataset['x1'];
x+
dataset = dataset.drop('x1', axis=1);
dataset = pd.get_dummies(dataset);
dataset.head()

,x3,x8,x9,target,x4_I,x4_L,x4_O,x4_S,x5_I,x5_L,x5_S,x6_I,x6_L,x6_O,x6_S,x7_O,x7_S
0,3914.25,-113,84,1,1,0,0,0,1,0,0,0,1,0,0,0,1
1,3915.25,-67,86,1,1,0,0,0,1,0,0,0,1,0,0,0,1
2,3915.75,-56,87,1,1,0,0,0,1,0,0,0,1,0,0,0,1
3,3915.50,-56,86,1,1,0,0,0,1,0,0,0,1,0,0,0,1
4,3916.00,-45,87,1,1,0,0,0,1,0,0,0,1,0,0,0,1


In [82]:
y = dataset['target'];
X = dataset.drop('target', axis=1);

In [83]:
## Creación modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [84]:
# Vamos a escalar los datos. 
# Crea el objeto y fitea
scaler = StandardScaler()
scaler.fit(X_train)

# normaliza los datos. tf: transformados o escalados
X_train_tf = scaler.transform(X_train)
X_test_tf = scaler.transform(X_test)

array([[-1.16895774, -0.04253203, -1.60064123, ...,  2.0902705 ,
        -0.59389661,  0.59389661],
       [ 0.01309966, -0.78295312,  0.16902231, ..., -0.47840698,
        -0.59389661,  0.59389661],
       [-1.04727536,  0.59211461, -1.43393379, ...,  2.0902705 ,
        -0.59389661,  0.59389661],
       ...,
       [-0.42148027,  1.59697181,  0.00231487, ..., -0.47840698,
        -0.59389661,  0.59389661],
       [-1.16895774,  0.204275  , -1.54934663, ...,  2.0902705 ,
        -0.59389661,  0.59389661],
       [ 0.3346888 ,  0.98876877,  0.69479191, ..., -0.47840698,
        -0.59389661,  0.59389661]])

## Random Forest

In [86]:
# Random Forest
# crea Classifier
rfc=RandomForestClassifier(n_estimators=300, criterion='entropy', min_samples_split=10)

# Entrena modelo y lo aplica para predecir
rfc.fit(X_train_tf,y_train)

y_train_pred_rf = rfc.predict(X_train_tf)
y_test_pred_rf = rfc.predict(X_test_tf)

print("Accuracy test: %.5f"%metrics.accuracy_score(y_test, y_test_pred_rf), '\n'
     'Accuracy train: %.5f'%metrics.accuracy_score(y_train, y_train_pred_rf))

Accuracy test: 0.99333 
Accuracy train: 0.99713


In [88]:
matriz_conf_rf = confusion_matrix(y_test, y_test_pred_rf)
matriz_conf_rf

array([[90,  1],
       [ 0, 59]], dtype=int64)